<a href="https://colab.research.google.com/github/pablohenrique93/projeto-pratico-python-machine-learning-2/blob/main/final_project_machine_learning_%7C%7C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação das Bibliotecas

In [ ]:
#Instalação do módulo de arima para se utilizar o algoritmo de previsão temporal
!pip install pmdarima

**Importação das Bibliotecas Padrões**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# Biblioteca padrão para trabalhar com datas
import datetime
# Decomposição de série temporal
from statsmodels.tsa.seasonal import seasonal_decompose
#Importando biblioteca para previsão temporal
from pmdarima.arima import auto_arima

#Carregamento da Base de Dados

In [3]:
#Lendo o arquivo de dados, já importando as colunas "ObservationDate" e "Last Update" em formato de data
#Para que o algoritmo possa entender e processar essas datas, e realizarmos previsões a partir disso.
df_covid= pd.read_csv(('/content/covid_19_data.csv'), parse_dates=['ObservationDate', 'Last Update'])

#Pré Análise dos Dados

In [ ]:
#Exibindo as primeiras linhas do dataset
df_covid.head(20)

In [ ]:
#Exibindo as ultimas linhas do dataset
df_covid.tail()

In [ ]:
#Exibição dos tipos das colunas do dataset
df_covid.dtypes

In [ ]:
#Exibição do tamanho
df_covid.shape

In [ ]:
#Exibindo algumas informações como o tipo de cada coluna e dados nulos
df_covid.info()

#Pré Visualização

**Como verificado, o dataset é muito grande, a partir disso optou-se por trabalhar com dados de apenas um país, sendo ele o Brasil**

In [9]:
#Filtro de dados utilizando o Brasil como parâmetro
df_covid_brasil = df_covid[df_covid['Country/Region']=='Brazil']

In [ ]:
#Exibindo o dataset
df_covid_brasil

In [ ]:
#Exibindo informações deste novo dataset
df_covid_brasil.info()
#É possível observar alguns dados nulos

In [ ]:
#Exibindo a quantidade de dados por estado
df_covid_brasil['Province/State'].value_counts().sort_values(ascending=False)
#É possível notar que os dados estão bem distribuídos entre os estados

In [ ]:
#Verificando a quantidade de valores nulos no dataset
df_covid_brasil.isnull().sum()
#Como verificado, temos 85 dados faltantes na coluna Province/State

#Tratamento dos Dados

In [ ]:
df_covid_brasil.head(1)

In [ ]:
#Percorrendo colunas com o objetivo de encontrar valores inconsistentes
pd.unique(df_covid_brasil['ObservationDate'])

In [ ]:
pd.unique(df_covid_brasil['Province/State'])

In [ ]:
pd.unique(df_covid_brasil['Country/Region'])

In [ ]:
pd.unique(df_covid_brasil['Last Update'])

In [ ]:
pd.unique(df_covid_brasil['Confirmed'])

In [ ]:
pd.unique(df_covid_brasil['Deaths'])

In [ ]:
pd.unique(df_covid_brasil['Recovered'])

# Limpeza de Inconsistências e Padronização de Colunas

In [ ]:
#Renomeação e tradução das colunas com o objetivo de facilitar o entendimento dos dados
df_covid_brasil.rename(columns={
    'SNo':'numero_de_serie',
    'ObservationDate':'data_da_observacao',
    'Province/State':'estado',
    'Country/Region':'pais',
    'Last Update':'ultima_atualizacao',
    'Confirmed':'casos_confirmados',
    'Deaths':'mortes_acumuladas',
    'Recovered':'casos_recuperados'
},inplace=True)

In [ ]:
#Conferindo dataset após alteração
df_covid_brasil

In [ ]:
#Traduzindo dados dentro das colunas "estado" e "país", para facilitar o entendimento
df_covid_brasil.replace({'estado':'Unknown'}, 'desconhecido',regex=True, inplace=True)
df_covid_brasil.replace({'pais':'Brazil'}, 'Brasil',regex=True, inplace=True)

#Visualização e Exploração dos Dados

In [ ]:
#Exibindo os casos confirmados por estado em porcentagem
casos_por_estado = df_covid_brasil.groupby(['estado'])['casos_confirmados'].sum().sort_values(ascending=False)
porcentagem_casos = (casos_por_estado / casos_por_estado.sum())*100
porcentagem_casos

In [ ]:
#Exibindo a análise anterior com auxílio de plot
porcentagem_casos.plot(kind='bar', figsize=(12,8))

In [ ]:
df_covid_brasil.dtypes

In [ ]:
#Exibindo a quantidade em porcentagem de casos de mortes por estado
mortes_por_estado = df_covid_brasil.groupby(['estado'])['mortes_acumuladas'].sum().sort_values(ascending=False)
porcentagem_mortes = (mortes_por_estado/mortes_por_estado.sum())*100
porcentagem_mortes

In [ ]:
#Exibindo a análise anterior com auxílio de plot
porcentagem_mortes.plot(kind='bar', figsize=(12,8))
#Como observado a maior parte dos casos de mortes vem de SP, RJ e MG

In [31]:
#Análise da taxa de recuperação por estado
recupera_por_estado = df_covid_brasil.groupby(['estado'])['casos_recuperados'].sum().sort_values(ascending=False)
tx_recupera = ((recupera_por_estado/casos_por_estado)*100).sort_values(ascending=False)

In [ ]:
#Exibição desta análise com auxílio de plot
tx_recupera.plot(kind='bar', figsize=(12,8))

In [33]:
# Analisando os casos ativos por estado
casos_ativos = (casos_por_estado - mortes_por_estado - recupera_por_estado).sort_values(ascending=False)
# Observa-se que muitas recuperações foram registradas como 'desconhecido', por isso está como negativo

In [ ]:
#Observando esta situação de maneira visual
casos_ativos.plot(kind='bar', figsize=(12,8))

In [ ]:
df_covid_brasil.dtypes

In [36]:
#Agregando os valores de casos confirmados, recuperados e mortes
df_brasil_casos =df_covid_brasil.groupby(["data_da_observacao"]).agg({"casos_confirmados":'sum',"casos_recuperados":'sum',"mortes_acumuladas":'sum'})

In [ ]:
#Plotando agora o gráfico de casos confirmados, recuperados e mortes ao longo do tempo
fig=go.Figure()
fig.add_trace(go.Scatter(x=df_brasil_casos.index, y=df_brasil_casos["casos_confirmados"], name='Confirmados'))
fig.add_trace(go.Scatter(x=df_brasil_casos.index, y=df_brasil_casos['casos_recuperados'], name='Recuperados'))

fig.add_trace(go.Scatter(x=df_brasil_casos.index, y=df_brasil_casos['mortes_acumuladas'], name='Mortes'))

fig.update_layout(title='Casos de Covid',
                   xaxis_title='data',
                   yaxis_title='Número de casos')
fig.show()

# Divisão da Base em Treino e Teste

In [38]:
#Divisão da base em treino e teste já selecionando o período/ De Fevereiro de 2020 a Abril de 2021 / A previsão será para Maio de 2021
treino = df_brasil_casos['2020-02-26': "2021-04-30"]
teste = df_brasil_casos["2021-05-01":]

In [39]:
#Selecionando apenas a coluna de casos confirmados para fazermos a previsão de casos confirmados
treino_previsao= treino['casos_confirmados']
teste_previsao = teste['casos_confirmados']
previsao_teste_treino = pd.concat([treino_previsao,teste_previsao],axis=0)

#Decomposição da Série

In [ ]:
#Exibindo o tamanho do dataset que será decomposto
len(treino_previsao)

In [43]:
#Criando a variável decomposição utilizando o seasonal_decompose que é uma biblioteca importada
#Vamos observar a tendência da sazonalidade e a aleatoriedade dos dados de treino de casos confirmados
#O modelo ultilizado será o multiplicative que assume que as séries temporais são o resultado 
#de uma multiplicação entre as componentes de tendência, sazonalidade e aleatoriedade
decomposicao = seasonal_decompose(treino_previsao, model='multiplicative')

In [44]:
#Criando 3 variáveis, a primeira é:
#tendencia: ou seja a tendencia que as coisas estão acontecendo
#sazonal: é a sazonalidade ou periodicidade
#aleatorio: aleatorio são dados que nao irão se encaixar em nenhum dos casos anteriores
tendencia = decomposicao.trend
sazonal = decomposicao.seasonal
aleatorio = decomposicao.resid

In [ ]:
#Plots
plt.subplots(figsize=(10, 7))
plt.subplot(411)
plt.plot(treino_previsao, label='Original')
plt.legend(loc='best')

#Observando de maneira visual a tendencia
plt.subplot(412)
plt.plot(tendencia, label='Tendencia')
plt.legend(loc='best')
#Podemos observar, temos uma tendencia de ascensão

#Vamos observar a sazonalidade ou periodicidade
plt.subplot(413)
plt.plot(sazonal, label='Sazonalidade')
plt.legend(loc='best')
#Como podemos ver temos picos de quedas e subidas na maioria dos períodos/
#Mas a média geral, como vimos no gráfico de tendencia, é de subida

#Exibição dos momentos aleatórios 
plt.subplot(414)
plt.plot(aleatorio, label='Aleatoriedade')
plt.legend(loc='best')
#São alguns momentos que não se enquadram nas situações anteriores
#Executando esse teste a grande diferença ficou na parte aleatória

**Testando com o model='additive'**

In [46]:
#O modelo desta vez será o 'additive'. que é um modelo que assume que as séries temporais são o resultado 
#da soma dos componentes de tendência, sazonalidade e aleatoriedade. Em outras palavras, cada observação é igual à soma dessas componentes.
decomposicao2 = seasonal_decompose(treino_previsao, model='additive')

In [47]:
#Criando as 3 variáveis:
tendencia2 = decomposicao2.trend
sazonal2 = decomposicao2.seasonal
aleatorio2 = decomposicao2.resid

In [ ]:
#Plots
plt.subplots(figsize=(10, 7))
plt.subplot(411)
plt.plot(treino_previsao, label='Original')
plt.legend(loc='best')

#Observando de maneira visual a tendencia
plt.subplot(412)
plt.plot(tendencia2, label='Tendencia')
plt.legend(loc='best')
#Podemos observar, temos uma tendencia de ascensão

#Vamos observar a sazonalidade ou periodicidade
plt.subplot(413)
plt.plot(sazonal2, label='Sazonalidade')
plt.legend(loc='best')
#Como podemos ver temos picos de quedas e subidas na maioria dos períodos,
#neste sentido foi bastante semelhante ao modelo anterior

#Exibição dos momentos aleatórios 
plt.subplot(414)
plt.plot(aleatorio2, label='Aleatoriedade')
plt.legend(loc='best')
#Através desta plotagem, é possível notar que as situações aleatórias são mais semelhantes
#a sazonalidade em alguns períodos como 2020/07 e 2021/01
#É possível concluir que em comparação ao gráfico do model "multiplicative" o model "additive" foi um pouco mais semelhante a periodicidade.

#Previsão com Algoritmo de ARIMA

In [49]:
#Prevendo o número de casos no próximo mês com ARIMA
#Nesta etapa o objetivo é fazer a previsão de casos para o próximo mês de Maio

In [ ]:
#Criação da variável modelo utilizando o auto_arima para sabermos os melhores parâmetros
modelo = auto_arima(treino_previsao, method= 'nm', max_D=2, stepwise=True, maxiter =50, trace=True)

In [ ]:
#Verificando o modelo que ele utilizou
modelo.order
#Como podemos observar, a ordem é 1, 2, 1

In [ ]:
#Aplicando os melhores parâmetros ao modelo
modelo.fit(treino_previsao)

In [53]:
#Como relatado anteriormente será realizado a previsão para os próximos 29 dias que são os dados do mês de Maio que temos
previsoes = modelo.predict(n_periods=29)

In [ ]:
#Essas são as previsões para os próximos 29 dias baseada no "treino_previsao"
previsoes

In [ ]:
#Tranformando as previsões em Dataframe para uma melhor visualização
previsoes=pd.DataFrame(previsoes,index=teste.index, columns=['previsão'])
previsoes

In [ ]:
#Verificando o tamanho do teste
teste_previsao.shape
#Como podemos observar o seu tamanho é 29, ou seja, 29 dias restantes do mês de Maio

In [ ]:
teste_previsao
#Comparando o nosso teste com as previsões

# Gráfico das Previsões

In [ ]:
#Vamos observar essa comparação de maneira visual
#Iremos verificar se os valores previstos são semelhantes aos valores originais
plt.figure(figsize=(20,5))
pd.concat([teste_previsao,previsoes],axis=1).plot(figsize=(20,5))
plt.xlabel('')
plt.title('Previsões X Original',size=15)
plt.xlabel("Período", size=15)
plt.legend(['Valor Original','Previsão'])

In [ ]:
#Observando a mesma situação, porém com a janela de tempo mais detalhada
pd.concat([df_brasil_casos['casos_confirmados'],previsoes],
          axis=1).plot(linewidth=2,
                       figsize=(20,7),
                       color=['g','r'])
plt.legend(['Casos de Covid','Previsões'], fontsize=15)
plt.xlabel('Período', size=15)
plt.title('Previsões X Real',size=15)
plt.show();


#Avaliação deste Algoritmo

In [ ]:
#Comparando o teste e a previsão através dos seus índices
teste_previsao[0:2]

In [ ]:
previsoes[0:2]

In [62]:
#Verificando a precisão de maneira automática, ou seja, cálculo para verificar o (Mean Absolute Percentage Error) MAPE
#Importação da biblioteca para realização do procedimento
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
mean_absolute_percentage_error(teste_previsao, previsoes)
#Como podemos verificar a porcentagem média de erro absoluto entre as previsões geradas pelo algoritmo e os 
#valores reais da série temporal é de aproximadamente 0,60%. Em outras palavras, o algoritmo tem uma precisão média de 99,40% 
#em relação aos valores reais. Quanto menor o valor do mean_absolute_percentage_error, melhor é a precisão do algoritmo. 
#Portanto, um MAPE de 0,60% indica que o algoritmo tem uma boa precisão em relação aos valores reais.

#Previsão com Algoritmo Facebook Prophet

In [ ]:
#Prevendo o número de casos para o mês de Maio com o Facebook Prophet

**Instalação e Importação das Bibliotecas** 

In [ ]:
#Instalação da biblioteca pystan que é uma interface Python para a linguagem Stan, que é uma linguagem de 
#modelagem estatística de alto desempenho.
#(Neste caso, foi necessário a instalação da mesma para a partir disso instalar a biblioteca do Facebook) 
!pip install pystan~=2.14

In [ ]:
#Instalação da biblioteca do Facebook Prophet
!pip install fbprophet

In [66]:
#Importação da mesma
from fbprophet import Prophet

In [ ]:
treino_previsao

In [68]:
#Inicialmente devemos renomear as colunas pois o algoritmo do Prophet 
#só identifica a coluna data se estiver nomeada como 'ds', e o valor que se deseja prever como "y"
treino_prophet =treino_previsao.reset_index()
treino_prophet = treino_prophet.rename(columns={'data_da_observacao' : 'ds', 'casos_confirmados': 'y'})

teste_prophet = teste_previsao.reset_index()
teste_prophet = teste_prophet.rename(columns={'data_da_observacao' : 'ds', 'casos_confirmados': 'y'})

prophet_teste_treino = pd.concat([treino_prophet,teste_prophet],axis=0)

In [ ]:
#Construção do modelo que será utilizado pelo algoritmo facebook prophet para realizar previsões
#Primeiro instanciamos o algoritmo 
modelo = Prophet()
#Depois usamos o algoritmo setando o nosso dataset
modelo.fit(treino_prophet)

In [ ]:
#Criação da variável 'futuro' setando o modelo juntamente com o período de 30 dias como no Arima
futuro = modelo.make_future_dataframe(periods= 30, freq = '1d')
#Depois criamos as previsões setando a nossa variável futuro
previsao = modelo.predict(futuro)

In [ ]:
#Como podemos observar ele cria muitas váriaveis contendo dados dos cálculos, algumas dessas colunas são termos aditivos, menor valor semanal,
# maior valor semanal, menor valor anual e etc, e a ultima coluna é a previsão 'yhat' 
previsao.head()

In [ ]:
#Verificando o tamanho do dataset e o tamanho das previsões
len(treino_prophet), len(previsao)

In [ ]:
#Conferindo se realmente está correto
len(previsao) - len(treino_prophet)
#Constatamos que está correto pois o retorno foi 30, ou seja, os 30 dias que colocamos para pesquisar

In [ ]:
#Exibindo as previsões de 30 dias
previsao.tail(30)

#Gráfico das Previsões

In [ ]:
#Plotando o gráfico de previsões
modelo.plot(previsao)

data_previsao = previsao[['ds', 'yhat']]
df_previsao = data_previsao.merge(teste_prophet, how='inner', on='ds')
#É possível observar através do gráfico, que o algoritmo Prophet foi bastante acertivo na maioria dos períodos,
#conseguindo acompanhar as variações durante os períodos, sendo mais acertivo do que o Algoritmo de Arima

In [ ]:
#Exibindo os componentes ou seja os dados de decomposição
modelo.plot_components(previsao);

In [77]:
#Utilizando o express para realizar estes modelos de forma mais dinâmica

In [78]:
#importação das bibliotecas
from fbprophet.plot import plot_plotly, plot_components_plotly

In [ ]:
#Plotagem do modelo e previsões de forma dinâmica, podendo classicar por semana, ano, 6 meses, todos e etc
plot_plotly(modelo, previsao)

In [ ]:
#Exibição dos componentes
plot_components_plotly(modelo, previsao)

#Avaliação deste Algoritmo

In [ ]:
#Calculando o erro MAPE (mean absolute error)
y_true = df_previsao['y']
y_pred = df_previsao['yhat']
mean_absolute_percentage_error(y_true, y_pred)
#Para este caso, o resultado encontrado com o Prophet foi melhor do que o resultado obtido com o ARIMA
#Valendo ressaltar que quanto menor a média de erro percentual em relação aos valores reais, 
#melhor a performance de previsão

#Conclusão

In [ ]:
'''
Ao chegar ao final deste projeto, é possível concluir que de forma geral os algoritmos de Arima e Facebook Prophet foram bem acertivos,
ou seja, em comparação com os valores reais eles obtiveram médias percentuais de erro baixas.

O MAPE mede a porcentagem média de erro em relação às observações reais e é uma métrica importante para avaliar a precisão 
dos modelos de previsão. Quanto menor o valor do MAPE, melhor é o desempenho do modelo.

Trazendo um comparativo entre os dois algoritmos, com base nos resultados obtidos, é possível afirmar que o modelo Facebook Prophet 
apresentou um desempenho superior em relação ao modelo ARIMA, uma vez que obteve-se um valor de 0.0007764758089230986 no cálculo 
do erro médio absoluto percentual (MAPE), enquanto o ARIMA obteve um valor de 0.005967400263522024. 

Portanto, considerando que o Facebook Prophet apresentou um MAPE significativamente menor em comparação ao ARIMA, 
pode-se afirmar que ele foi o modelo mais acertivo na previsão dos dados analisados. É importante ressaltar que, 
embora o Facebook Prophet tenha apresentado melhor desempenho neste caso, é sempre recomendado avaliar diferentes modelos 
e métricas de avaliação para escolher a melhor abordagem de acordo com as características do problema em questão.

'''